### Enter variables

In [26]:
# Input files
storage_shapefile = "BR_Sample_Parcels_proj.shp"
storage_id_field = "qldglobe_p"

# Output files
outputCSV = "GEE_Map_Cropping_Sentinel.csv"

# Cloud cover
CLOUD_FILTER = 0 # Under 40%

# Thresholds
ndwiMaskVal = -0.1
ndviMaskVal = 0.5

# UTM Zone
utmZoneInfo = 'EPSG:28355'#z55S

# Dates
start_date = '2022-01-01'
end_date = '2022-01-30'

### Import modules

In [27]:
import ee, geemap, folium, pandas, geopandas as gpd, json, datetime, altair as alt, csv
from IPython.display import Image
import ee.mapclient
import datetime
import os
from os.path import exists as file_exists

import requests

# importing pandas as pd
import pandas as pd

### Declare functions

In [29]:
def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')

def addDate(image):
    # parse date stored in 'system:index'
    date = ee.Date(image.get('system:index'))

    # format date, see http:#www.joda.org/joda-time/apidocs/org/joda/time/format/DateTimeFormat.html
    str = date.format('YYYY-mm-dd')

    return image.set({'Date': str})

def mean_func(image):
    mean = (image                        
            #.mean()
            .reduceRegion(ee.Reducer.mean()))#, storage_geometry, 30))
    mean = mean.getInfo()            
    mean = mean.get('nd')    
    return mean

def test_api():
    dem = ee.Image('USGS/SRTMGL1_003')
    xy = ee.Geometry.Point([86.9250, 27.9881])
    elev = dem.sample(xy, 30).first().get('elevation').getInfo()
    print('Authentication active')
    
def transformer(feature):
    transformed_feature = feature.transform(utmZoneInfo, 0.001);
    return transformed_feature;

def filterImages(storage_geometry):
    # Load Sentinel2
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(storage_geometry)
        .filterDate(start_date, end_date))
        #.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))
    
    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(storage_geometry)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }));



def exportStats(lot):
    # Create a new dataframe
    storageStats = pandas.DataFrame(columns=['Lotplan', 'Date', 'Total Area', 'Veg Area', 'Cloud Cover'])        
    # Select the feature
    storage_selection = storage_collection.filter(ee.Filter.eq(storage_id_field, lot))   
    # Get the geometry
    storage_geometry = storage_selection.geometry()
    # Create list of images
    images = filterImages(storage_geometry)
    # Clip images to storage and reproject
    images = images.map(lambda image: image.clip(storage_geometry).reproject(crs=utmZoneInfo))
    # Create a list of images
    imageSize = images.size().getInfo()    
    imageList = images.toList(images.size()) 
    # Get stats ##
    # For each image
    for img in range(0, imageSize):    
        # Choose the image
        image = ee.Image(imageList.get(img))
        # Get date
        date = ee.Date(image.get('system:time_start')).format("yyyy-MM-dd")
        date = date.getInfo()
        print(date)
        # Create images
        ndvi = image.normalizedDifference(['B8', 'B4'])
        ndviMaskLayer = ndvi.gte(ndviMaskVal).rename('ndviMaskLayer')
        #This would retain values of NDWI, not set them to 0/1
        ndvimasked = ndvi.updateMask(ndviMaskLayer)#.rename('ndwi')
        #Need one complete raster for total area stats calc
        ndviProj = ndvi.reproject(crs=utmZoneInfo, scale=10)#Chosen 10 metres, , crsTransform='null'
        ndviMaskedProj = ndviMaskLayer.reproject(crs=utmZoneInfo, scale=10)
        storage_area = ndviProj.gte(-50.0).multiply(ee.Image.pixelArea()).rename('allPixels')
        veg_area = ndviMaskedProj.multiply(ee.Image.pixelArea()).rename('ndwiStatsLayer')
        

        
        
        # Get cloud cover
        cloudiness = image.get("CLOUD_COVER").getInfo()
        # Add results to storage stats dataframe
        results = [lot,date,storage_area,veg_area,cloudiness]
        print(results)
        storageStats.loc[len(storageStats)] = results  
        storageStatsMaster.loc[len(storageStats)] = results   
        # Export image
        #exportRGBImage(image)
    # Save to CSV
    storageStats.to_csv(outputCSV)

### Authenticate Earth Engine

In [22]:
# Test access then authenticate if fails
try:
    test_api()
except:    
    ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Authentication active


### Convert shapefile into feature collection

In [23]:
# Load shapefile into geopandas
storage_shapes = gpd.read_file(storage_shapefile)

# Convert shapes into json
json_storages = json.loads(storage_shapes.to_json())

# Convert json into feature collection
storage_collection =  ee.FeatureCollection(json_storages)

# Reproject collection
#storage_collection = storage_collection.map(transformer);

# Add area
storage_collection.map(lambda feature: feature.set('total_area', ee.Number(feature.area()).divide(1e6).round()))

### Get a list of Storage IDs

In [24]:
# list the unique quality code values
worksSet = sorted(storage_shapes[storage_id_field].unique())
print(worksSet)

['101SP207166', '108SP207166', '110F6901', '111F6901', '11RP218867', '14CVN407', '15CVN386', '16BLM76', '17CVN407', '1BLM66', '1BLM846', '1PER4265', '1RP225469', '1SP132773', '236RP851344', '23SP225448', '24PG406', '25PG260', '297SP146073', '2BLM1', '2BLM465', '2SP276749', '31BLM809', '33CP849174', '39BLM782', '3BLM1', '3RP225469', '3SP276749', '4BEL5359', '4SP276749', '5RP212965', '5RP213579', '5SP225462', '5SP276749', '612RP909550', '62SP225462', '63CP858891', '6RP213579', '71RP213572', '73RP213572', '7BLM715', '8CVN140', 'AAP21199']


### Clip imagery to each storage

In [ ]:
# Create a new dataframe
storageStatsMaster = pandas.DataFrame(columns=['Lotplan', 'Date', 'Total Area', 'Veg Area', 'Cloud Cover'])  

# Start timer
startNow = datetime.datetime.now()
print("Starting at " + str(startNow))

# For each storage
for worksNumber in worksSet:
    
    # Create file name for csv export
    outputCSV = worksNumber + "_Sentinel_Timeseries.csv"
    
    # Check if already completed
    if file_exists(outputCSV):
        
        print("Skipping " + worksNumber)
        
    else:
        
        # Create an image function to map
        print("Starting " + worksNumber)
        exportStats(worksNumber)
        

# Save master     
storageStats.to_csv(outputCSV)
        
# Print the storage stats
print("Finished")

# Finish time
endNow = datetime.datetime.now()
diff = (endNow-startNow).total_seconds()
print("End at " + str(endNow))
print("Taking: " + str(diff) + " seconds")

Starting at 2023-03-03 14:08:59.960408
Starting 101SP207166
2022-01-01
['101SP207166', '2022-01-01', <ee.image.Image object at 0x000002098A603490>, <ee.image.Image object at 0x0000020989C2D790>, None]
2022-01-01
['101SP207166', '2022-01-01', <ee.image.Image object at 0x000002098A16A220>, <ee.image.Image object at 0x000002098A16A910>, None]
2022-01-01
